### Import packages

Import of the packages that will be needed for the project.  This includes packages for data manipulation, sklearn modules and custom functions.

In [3]:
import pandas as pd
import numpy as np
import pickle

import sklearn
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder, RobustScaler, StandardScaler, MinMaxScaler, FunctionTransformer

from Project_Functions import offensive_contribution, trailing_stats_mean, tier_maker, get_tiers, get_touchdowns, stats_for_trailing

In [4]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action = 'ignore',
                     category = SettingWithCopyWarning)

### Import Data

Let's import the dataframe that we will be using for modelling

In [5]:
data = pd.read_csv('Data/weekly_data.csv')

data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade40to49,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0


In [6]:
data.isna().sum()

Unnamed: 0      0
Unnamed: 0.1    0
GameKey         0
PlayerID        0
Season          0
               ..
HomePoints      0
HomeYards       0
AwayTeam        0
AwayPoints      0
AwayYards       0
Length: 65, dtype: int64

Before the train test split, we have to calculate the trailing average fantasy points for each observation, as we cannot incorporate this step into the pipeline without causing data leakage.

In [7]:
def trailing_stats(df):
    
    """
    Function to create a new column with a trailing aggregate mean
    as a new feature for prediction.
    
    Inputs:
        - df: The dataframe on which the function will be applied
        - Column: The column on which to apply the function
        - Window: The number of past values to consider when apply the function

        
    Output:
        - An aggregate value
        
    """
    #Access the column names in stats_for_trailing
    global stats_for_trailing
    
    # Get all unique players in the DataFrame
    players = df['Name'].unique().tolist()
    
    # Define a DataFrame to hold our values
    df_out = pd.DataFrame()
    # Loop through the unique players
    for player in players:
        # Create a temporary dataframe for each player
        temp_df = df[df['Name'] == player]
        # Calculate the n game trailing average for all players.  Set closed parameter to 'left'
        # so that the current value for fantasy points is not included in the calculation.
        # Backfill the two resulting NaN values
        for column in stats_for_trailing:
            temp_df[f'TA{column}'] = temp_df.loc[:,column].rolling(window = 5, 
                                                              closed = 'left').mean().fillna(method = 'bfill')
        # Append the temporary dataframe to the output
        df_out = df_out.append(temp_df)
    # Return a dataframe with the values sorted by the original index
    return df_out.sort_index()
    

In [8]:
stats_for_trailing = ['FantasyPointsPPR']

In [9]:
# Prepare the trailing average fantasy points column

data = trailing_stats(data)
data.head()

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,FieldGoalsMade50Plus,InjuryStatus,MatchString,HomeTeam,HomePoints,HomeYards,AwayTeam,AwayPoints,AwayYards,TAFantasyPointsPPR
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,20.088
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,28.600
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,0.0,NaN,20181TBNO,NO,40.0,475.0,TB,48.0,529.0,25.204
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,0.0,NaN,20181KCLAC,LAC,28.0,541.0,KC,38.0,362.0,18.240
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,0.0,NaN,20181PITCLE,CLE,21.0,327.0,PIT,21.0,472.0,22.420


### Train Test Split

In [10]:
# Separate data from the target
# y = data['FantasyPointsPPR']

# Apply a log transform to the target to see how it impacts prediction
y = data['FantasyPointsPPR']
data.drop(columns = ['FantasyPointsPPR'],
            inplace = True)
X = data

In [11]:
# Execute the train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                   test_size = 0.25,
                                                   random_state = 13)

### Feature Engineering

The main features that we will be engineering to predict a player's fantasy output will be the 5-game trailing average of various statistics as well as the binning of players into their respective tiers based on recent performance.

In [12]:
# Define the columns for which we want a 5 game trailing average.

stats_for_trailing = ['TotalTouchdowns','RushingYards','PassingInterceptions','PassingTouchdowns','PassingRating','PassingYards',
                      'PassingCompletionPercentage','RushingYards', 'RushingTouchdowns', 'RushingLong',
                      'Receptions','ReceivingYards','ReceivingTargets', 'ReceivingTouchdowns', 'ExtraPointsMade', 'FieldGoalsMade',
                      'Fumbles','FumblesLost']

trailing_stats = []
for col in stats_for_trailing:
    trailing_stats.append('TA' + col)
trailing_stats

['TATotalTouchdowns',
 'TARushingYards',
 'TAPassingInterceptions',
 'TAPassingTouchdowns',
 'TAPassingRating',
 'TAPassingYards',
 'TAPassingCompletionPercentage',
 'TARushingYards',
 'TARushingTouchdowns',
 'TARushingLong',
 'TAReceptions',
 'TAReceivingYards',
 'TAReceivingTargets',
 'TAReceivingTouchdowns',
 'TAExtraPointsMade',
 'TAFieldGoalsMade',
 'TAFumbles',
 'TAFumblesLost']

In [13]:
# Instantiate the function transformers for the feature engineering pipeline

touchdown_transformer = FunctionTransformer(get_touchdowns) # Get total touchdowns per player
trailing_transformer = FunctionTransformer(trailing_stats_mean) # Get the 5 game trailing averages of appropriate statistics
tier_transformer = FunctionTransformer(get_tiers) # Bin players into the appropriate tiers based on recent performance


# Instantiate the pipeline for the necessary transformations

engineering = Pipeline([('touchdown', touchdown_transformer),
                       ('trailing', trailing_transformer),
                       ('tier', tier_transformer)])

In [14]:
df_test = engineering.fit_transform(X_train)

In [15]:
df_test

,Unnamed: 0,Unnamed: 0.1,GameKey,PlayerID,Season,GameDate,Week,Team,Opponent,HomeOrAway,...,TARushingLong,TAReceptions,TAReceivingYards,TAReceivingTargets,TAReceivingTouchdowns,TAExtraPointsMade,TAFieldGoalsMade,TAFumbles,TAFumblesLost,PlayerTier
0,0,0,201810122,8283,2018,2018-09-09T13:00:00,1,TB,NO,AWAY,...,11.4,0.0,0.0,0.0,0.0,0.0,0.0,0.4,0.2,QB2
1,1,1,201810122,18878,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,9.4,4.6,32.4,5.8,0.0,0.0,0.0,0.0,0.0,RB1
2,2,2,201810122,7242,2018,2018-09-09T13:00:00,1,NO,TB,HOME,...,2.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,QB1
3,4,4,201810129,18082,2018,2018-09-09T16:05:00,1,KC,LAC,AWAY,...,7.6,6.8,112.4,10.2,1.2,0.0,0.0,0.0,0.0,WR1
4,5,5,201810108,18983,2018,2018-09-09T13:00:00,1,PIT,CLE,AWAY,...,13.4,2.6,33.0,3.0,0.0,0.0,0.0,0.2,0.0,RB1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31530,101813,1517,202111031,21801,2021,2021-11-15T20:15:00,10,SF,LAR,HOME,...,3.2,0.4,0.6,0.6,0.0,0.0,0.0,0.0,0.0,RB4
31531,101815,1519,202111031,22119,2021,2021-11-15T20:15:00,10,SF,LAR,HOME,...,0.0,0.4,6.0,0.4,0.0,0.0,0.0,0.0,0.0,TE3
31533,101821,1525,202111025,2405,2021,2021-11-14T20:20:00,10,KC,LV,AWAY,...,0.8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,QB4
31535,101823,1527,202111012,22569,2021,2021-11-14T16:25:00,10,SEA,GB,AWAY,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,WR4


<br>

### Preprocessing

As shown above, the bulk of the null values fall into one of two categories.  They are either:
* In the InjuryStatus column
    * Here we can impute a value of healthy, as the only values in the injury column are 
* In the TA (trailing average) columns we created
    * No player with a null value played more than 5 games, therefore we cannot calculate the trailing average for them.  We will impute a default value of 0 for these columns, as they represent players who likely did not have much impact.  If they had an impact, they likely would have played in more games.  I will explore imputing the median value as well through a grid search.

In [12]:
# Define the groups of columns for preprocessing steps.

categorical_columns = ['Week',
                       'Team',
                       'Name',
                      'Opponent',
                      'PlayerTier',
                      'InjuryStatus']
numerical_columns = trailing_stats


In [13]:
# Define the preprocessing steps for categorical features
categorical_transform = Pipeline([('impute_cat',SimpleImputer(strategy = 'constant',
                                                              fill_value = 'Healthy')),
                                 ('one_hot_encoder', OneHotEncoder(handle_unknown = 'ignore'))])

# Define the preprocessing steps for numerical features
numerical_transform = Pipeline([('impute_num', SimpleImputer(strategy = 'constant',
                                                             fill_value = 0)),
                               ('scaler', RobustScaler()),
                               ('PCA', PCA())])

# Instantiate the column transformer object for the preprocessing steps
preprocessing = ColumnTransformer([('num', numerical_transform, numerical_columns),
                                  ('cat', categorical_transform, categorical_columns)])

In [14]:
# Instantiate a pipeline with a linear regression model as a baseline

pipeline = Pipeline([('engineering', engineering),
                    ('prep', preprocessing),
                    ('model', LinearRegression())])

In [15]:
# Set param grid values, parameters for grid search 

param_grid = {'prep__num__PCA__n_components': [5, 10, 15]}

grid_search = GridSearchCV(pipeline, 
                          param_grid = param_grid,
                           scoring = 'neg_mean_absolute_error',
                          cv = 5,
                          verbose = 3)

In [16]:
# Fit the grid search to X_train and y_train

grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 1/5] END ...prep__num__PCA__n_components=5;, score=-6.111 total time=  37.5s
[CV 2/5] END ...prep__num__PCA__n_components=5;, score=-6.066 total time=  36.1s
[CV 3/5] END ...prep__num__PCA__n_components=5;, score=-6.144 total time=  35.4s
[CV 4/5] END ...prep__num__PCA__n_components=5;, score=-6.151 total time=  35.8s
[CV 5/5] END ...prep__num__PCA__n_components=5;, score=-6.170 total time=  35.8s
[CV 1/5] END ..prep__num__PCA__n_components=10;, score=-6.106 total time=  36.4s
[CV 2/5] END ..prep__num__PCA__n_components=10;, score=-6.060 total time=  36.2s
[CV 3/5] END ..prep__num__PCA__n_components=10;, score=-6.151 total time=  37.6s
[CV 4/5] END ..prep__num__PCA__n_components=10;, score=-6.154 total time=  38.0s
[CV 5/5] END ..prep__num__PCA__n_components=10;, score=-6.176 total time=  41.3s
[CV 1/5] END ..prep__num__PCA__n_components=15;, score=-6.104 total time=  40.9s
[CV 2/5] END ..prep__num__PCA__n_components=15;, 

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('engineering',
                                        Pipeline(steps=[('touchdown',
                                                         FunctionTransformer(func=<function get_touchdowns at 0x7f87760419d0>)),
                                                        ('trailing',
                                                         FunctionTransformer(func=<function trailing_stats_mean at 0x7f8776041820>)),
                                                        ('tier',
                                                         FunctionTransformer(func=<function get_tiers at 0x7f8776041940>))])),
                                       ('prep',
                                        ColumnTransformer(transform...
                                                                        ('cat',
                                                                         Pipeline(steps=[('impute_cat',
                             

In [17]:
grid_search.best_score_

-6.126786519001913

In [18]:
grid_search.best_params_

{'prep__num__PCA__n_components': 15}

In [19]:
preds = grid_search.predict(X_test)

In [20]:
mean_squared_error(y_test, preds)

62.44954025114007

In [21]:
mean_absolute_error(y_test, preds)

6.080369620337555

Let's dig into the predictions a little bit

In [22]:
df_preds = pd.DataFrame(y_test)

In [23]:
df_preds['Predicted'] = preds

In [24]:
df_preds['PredictedReal'] = np.exp(df_preds['Predicted'])
df_preds['FPPRReal'] = np.exp(df_preds['FantasyPointsPPR'])
df_preds

,FantasyPointsPPR,Predicted,PredictedReal,FPPRReal
22597,9.0,2.698384,14.855702,8.103084e+03
30981,0.0,7.997170,2972.534382,1.000000e+00
14444,19.5,4.302820,73.907943,2.942676e+08
20883,0.0,2.913729,18.425386,1.000000e+00
21136,9.8,3.494160,32.922625,1.803374e+04
...,...,...,...,...
7713,0.0,5.669591,289.916032,1.000000e+00
9304,0.0,5.391507,219.533962,1.000000e+00
22562,16.0,7.669863,2142.788296,8.886111e+06
21120,6.8,3.737399,41.988643,8.978473e+02


### Save model


In [25]:
import pickle

pickle.dump(grid_search, open('Pickles/linreg_PCA.pickle', 'wb'))